In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.market_basket_dataset import MarketBasketDataset, BinaryEncodingTransform, RemoveItemsTransform
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [3]:
k = 8
tree_depth = 6
device = 'cuda'
dataset_path = r"/mnt/qnap/ekosman/Groceries_dataset.csv"

## Create the market basket dataset and use one-hot encoding for items

In [4]:
dataset = MarketBasketDataset(dataset_path=dataset_path)

In [5]:
model = AutoEncoder(dataset.n_items, 50, 4).train().to(device)
epochs = 500
lr = 5e-3
batch_size = 512
log_every = 5

In [6]:
dataset.transform = torchvision.transforms.Compose([
    RemoveItemsTransform(p=0.5),
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [7]:
model.train()
data_iter = torch.utils.data.DataLoader(dataset,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=1,
                                     pin_memory=True)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=0.7, threshold=1e-4)
knn_crt = KNNLoss(k=k).to(device)
losses = []
alpha = 10/170
gamma = 2
for epoch in range(epochs):
    total_loss = 0
    for iteration, (batch, target) in enumerate(data_iter):
        batch = batch.to(device)
        target = target.to(device)
        outputs, iterm = model(batch, return_intermidiate=True)
        mse_loss = F.binary_cross_entropy_with_logits(outputs, target, reduction='none')
        mask = torch.ones_like(mse_loss)
        mask[target == 0] = alpha ** gamma
        mask[target == 1] = (1 - alpha) ** gamma
        mse_loss = (mse_loss * mask).sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(iterm)
            if torch.isinf(knn_loss):
                knn_loss = 0
        except ValueError:
            knn_loss = torch.tensor(0)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(data_iter)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | BCE Loss: {mse_loss.item()}")
    
    scheduler.step(total_loss / (iteration + 1))
    losses.append(total_loss / (iteration + 1))

Epoch 0 / 500 | iteration 0 / 30 | Total Loss: 8.22247314453125 | KNN Loss: 6.22360897064209 | BCE Loss: 1.998863935470581
Epoch 0 / 500 | iteration 5 / 30 | Total Loss: 8.218988418579102 | KNN Loss: 6.223231792449951 | BCE Loss: 1.9957571029663086
Epoch 0 / 500 | iteration 10 / 30 | Total Loss: 8.13967514038086 | KNN Loss: 6.222769737243652 | BCE Loss: 1.9169052839279175
Epoch 0 / 500 | iteration 15 / 30 | Total Loss: 8.164443969726562 | KNN Loss: 6.222145080566406 | BCE Loss: 1.9422986507415771
Epoch 0 / 500 | iteration 20 / 30 | Total Loss: 8.195916175842285 | KNN Loss: 6.222169399261475 | BCE Loss: 1.9737468957901
Epoch 0 / 500 | iteration 25 / 30 | Total Loss: 8.157309532165527 | KNN Loss: 6.221463680267334 | BCE Loss: 1.9358460903167725
Epoch 1 / 500 | iteration 0 / 30 | Total Loss: 8.15555191040039 | KNN Loss: 6.221240043640137 | BCE Loss: 1.934312105178833
Epoch 1 / 500 | iteration 5 / 30 | Total Loss: 8.12782096862793 | KNN Loss: 6.2208991050720215 | BCE Loss: 1.90692198276519

Epoch 10 / 500 | iteration 25 / 30 | Total Loss: 5.264458656311035 | KNN Loss: 4.099384307861328 | BCE Loss: 1.165074110031128
Epoch 11 / 500 | iteration 0 / 30 | Total Loss: 5.162677764892578 | KNN Loss: 4.019097805023193 | BCE Loss: 1.1435799598693848
Epoch 11 / 500 | iteration 5 / 30 | Total Loss: 5.036418914794922 | KNN Loss: 3.8690450191497803 | BCE Loss: 1.1673741340637207
Epoch 11 / 500 | iteration 10 / 30 | Total Loss: 4.942875862121582 | KNN Loss: 3.8023130893707275 | BCE Loss: 1.1405625343322754
Epoch 11 / 500 | iteration 15 / 30 | Total Loss: 4.766353607177734 | KNN Loss: 3.647848129272461 | BCE Loss: 1.1185054779052734
Epoch 11 / 500 | iteration 20 / 30 | Total Loss: 4.7572808265686035 | KNN Loss: 3.615638494491577 | BCE Loss: 1.141642451286316
Epoch 11 / 500 | iteration 25 / 30 | Total Loss: 4.646496772766113 | KNN Loss: 3.518486499786377 | BCE Loss: 1.1280100345611572
Epoch 12 / 500 | iteration 0 / 30 | Total Loss: 4.577668190002441 | KNN Loss: 3.443626880645752 | BCE Los

Epoch 21 / 500 | iteration 15 / 30 | Total Loss: 3.7771406173706055 | KNN Loss: 2.7218680381774902 | BCE Loss: 1.0552725791931152
Epoch 21 / 500 | iteration 20 / 30 | Total Loss: 3.9323348999023438 | KNN Loss: 2.8805902004241943 | BCE Loss: 1.0517446994781494
Epoch 21 / 500 | iteration 25 / 30 | Total Loss: 3.7543439865112305 | KNN Loss: 2.7042951583862305 | BCE Loss: 1.0500487089157104
Epoch 22 / 500 | iteration 0 / 30 | Total Loss: 3.789013385772705 | KNN Loss: 2.7432188987731934 | BCE Loss: 1.0457944869995117
Epoch 22 / 500 | iteration 5 / 30 | Total Loss: 3.732511520385742 | KNN Loss: 2.7018165588378906 | BCE Loss: 1.0306950807571411
Epoch 22 / 500 | iteration 10 / 30 | Total Loss: 3.727170705795288 | KNN Loss: 2.662597417831421 | BCE Loss: 1.0645732879638672
Epoch 22 / 500 | iteration 15 / 30 | Total Loss: 3.7364912033081055 | KNN Loss: 2.7041547298431396 | BCE Loss: 1.0323365926742554
Epoch 22 / 500 | iteration 20 / 30 | Total Loss: 3.8142006397247314 | KNN Loss: 2.73337197303771

Epoch 32 / 500 | iteration 5 / 30 | Total Loss: 3.669727325439453 | KNN Loss: 2.654287099838257 | BCE Loss: 1.0154403448104858
Epoch 32 / 500 | iteration 10 / 30 | Total Loss: 3.717719554901123 | KNN Loss: 2.6808359622955322 | BCE Loss: 1.0368835926055908
Epoch 32 / 500 | iteration 15 / 30 | Total Loss: 3.7720143795013428 | KNN Loss: 2.7191321849823 | BCE Loss: 1.052882194519043
Epoch 32 / 500 | iteration 20 / 30 | Total Loss: 3.696995735168457 | KNN Loss: 2.63246750831604 | BCE Loss: 1.0645281076431274
Epoch 32 / 500 | iteration 25 / 30 | Total Loss: 3.6508092880249023 | KNN Loss: 2.6109132766723633 | BCE Loss: 1.039896011352539
Epoch 33 / 500 | iteration 0 / 30 | Total Loss: 3.650197982788086 | KNN Loss: 2.6085948944091797 | BCE Loss: 1.0416029691696167
Epoch 33 / 500 | iteration 5 / 30 | Total Loss: 3.7031362056732178 | KNN Loss: 2.636103868484497 | BCE Loss: 1.0670323371887207
Epoch 33 / 500 | iteration 10 / 30 | Total Loss: 3.753002643585205 | KNN Loss: 2.708535671234131 | BCE Los

Epoch 42 / 500 | iteration 25 / 30 | Total Loss: 3.645094156265259 | KNN Loss: 2.617058277130127 | BCE Loss: 1.0280358791351318
Epoch 43 / 500 | iteration 0 / 30 | Total Loss: 3.6482696533203125 | KNN Loss: 2.6339664459228516 | BCE Loss: 1.0143030881881714
Epoch 43 / 500 | iteration 5 / 30 | Total Loss: 3.686985492706299 | KNN Loss: 2.6422319412231445 | BCE Loss: 1.0447534322738647
Epoch 43 / 500 | iteration 10 / 30 | Total Loss: 3.5824618339538574 | KNN Loss: 2.5724573135375977 | BCE Loss: 1.0100046396255493
Epoch 43 / 500 | iteration 15 / 30 | Total Loss: 3.63242769241333 | KNN Loss: 2.593766927719116 | BCE Loss: 1.0386606454849243
Epoch 43 / 500 | iteration 20 / 30 | Total Loss: 3.6196696758270264 | KNN Loss: 2.5609395503997803 | BCE Loss: 1.058730125427246
Epoch 43 / 500 | iteration 25 / 30 | Total Loss: 3.5884146690368652 | KNN Loss: 2.5718188285827637 | BCE Loss: 1.0165958404541016
Epoch 44 / 500 | iteration 0 / 30 | Total Loss: 3.5769121646881104 | KNN Loss: 2.5403707027435303 |

...

Epoch 455 / 500 | iteration 5 / 30 | Total Loss: 3.497344493865967 | KNN Loss: 2.4596242904663086 | BCE Loss: 1.0377200841903687
Epoch 455 / 500 | iteration 10 / 30 | Total Loss: 3.4561691284179688 | KNN Loss: 2.4461653232574463 | BCE Loss: 1.0100038051605225
Epoch 455 / 500 | iteration 15 / 30 | Total Loss: 3.4925224781036377 | KNN Loss: 2.4930953979492188 | BCE Loss: 0.999427080154419
Epoch 455 / 500 | iteration 20 / 30 | Total Loss: 3.4616498947143555 | KNN Loss: 2.4440789222717285 | BCE Loss: 1.0175708532333374
Epoch 455 / 500 | iteration 25 / 30 | Total Loss: 3.4563753604888916 | KNN Loss: 2.427299737930298 | BCE Loss: 1.0290756225585938
Epoch 456 / 500 | iteration 0 / 30 | Total Loss: 3.45928955078125 | KNN Loss: 2.4431700706481934 | BCE Loss: 1.016119360923767
Epoch 456 / 500 | iteration 5 / 30 | Total Loss: 3.51631498336792 | KNN Loss: 2.4851460456848145 | BCE Loss: 1.0311689376831055
Epoch 456 / 500 | iteration 10 / 30 | Total Loss: 3.4680538177490234 | KNN Loss: 2.42786955833

Epoch 465 / 500 | iteration 20 / 30 | Total Loss: 3.423213481903076 | KNN Loss: 2.429776906967163 | BCE Loss: 0.9934365153312683
Epoch 465 / 500 | iteration 25 / 30 | Total Loss: 3.462337017059326 | KNN Loss: 2.4411447048187256 | BCE Loss: 1.0211924314498901
Epoch 466 / 500 | iteration 0 / 30 | Total Loss: 3.4683661460876465 | KNN Loss: 2.444000005722046 | BCE Loss: 1.0243662595748901
Epoch 466 / 500 | iteration 5 / 30 | Total Loss: 3.4846179485321045 | KNN Loss: 2.460521697998047 | BCE Loss: 1.0240962505340576
Epoch 466 / 500 | iteration 10 / 30 | Total Loss: 3.4645864963531494 | KNN Loss: 2.4379167556762695 | BCE Loss: 1.0266697406768799
Epoch 466 / 500 | iteration 15 / 30 | Total Loss: 3.4365944862365723 | KNN Loss: 2.4429233074188232 | BCE Loss: 0.9936712980270386
Epoch 466 / 500 | iteration 20 / 30 | Total Loss: 3.4956912994384766 | KNN Loss: 2.4752302169799805 | BCE Loss: 1.0204612016677856
Epoch 466 / 500 | iteration 25 / 30 | Total Loss: 3.4406561851501465 | KNN Loss: 2.4264559

Epoch 476 / 500 | iteration 5 / 30 | Total Loss: 3.445772647857666 | KNN Loss: 2.4483275413513184 | BCE Loss: 0.9974452257156372
Epoch 476 / 500 | iteration 10 / 30 | Total Loss: 3.5029916763305664 | KNN Loss: 2.4645087718963623 | BCE Loss: 1.038482904434204
Epoch 476 / 500 | iteration 15 / 30 | Total Loss: 3.471266984939575 | KNN Loss: 2.469269275665283 | BCE Loss: 1.001997709274292
Epoch 476 / 500 | iteration 20 / 30 | Total Loss: 3.5171613693237305 | KNN Loss: 2.491731882095337 | BCE Loss: 1.025429606437683
Epoch 476 / 500 | iteration 25 / 30 | Total Loss: 3.455288887023926 | KNN Loss: 2.4493396282196045 | BCE Loss: 1.0059492588043213
Epoch 477 / 500 | iteration 0 / 30 | Total Loss: 3.4545929431915283 | KNN Loss: 2.4520113468170166 | BCE Loss: 1.0025815963745117
Epoch 477 / 500 | iteration 5 / 30 | Total Loss: 3.4612174034118652 | KNN Loss: 2.428755283355713 | BCE Loss: 1.0324621200561523
Epoch 477 / 500 | iteration 10 / 30 | Total Loss: 3.4868459701538086 | KNN Loss: 2.441899061203

Epoch 486 / 500 | iteration 20 / 30 | Total Loss: 3.461009979248047 | KNN Loss: 2.439967632293701 | BCE Loss: 1.0210423469543457
Epoch 486 / 500 | iteration 25 / 30 | Total Loss: 3.4619665145874023 | KNN Loss: 2.424189329147339 | BCE Loss: 1.037777304649353
Epoch 487 / 500 | iteration 0 / 30 | Total Loss: 3.4783449172973633 | KNN Loss: 2.447495937347412 | BCE Loss: 1.0308489799499512
Epoch 487 / 500 | iteration 5 / 30 | Total Loss: 3.4994096755981445 | KNN Loss: 2.479006290435791 | BCE Loss: 1.0204033851623535
Epoch 487 / 500 | iteration 10 / 30 | Total Loss: 3.473900079727173 | KNN Loss: 2.4573276042938232 | BCE Loss: 1.0165724754333496
Epoch 487 / 500 | iteration 15 / 30 | Total Loss: 3.4627180099487305 | KNN Loss: 2.4414563179016113 | BCE Loss: 1.0212615728378296
Epoch 487 / 500 | iteration 20 / 30 | Total Loss: 3.4927616119384766 | KNN Loss: 2.458444118499756 | BCE Loss: 1.0343174934387207
Epoch 487 / 500 | iteration 25 / 30 | Total Loss: 3.4832425117492676 | KNN Loss: 2.4796440601

Epoch 497 / 500 | iteration 5 / 30 | Total Loss: 3.503479480743408 | KNN Loss: 2.483837842941284 | BCE Loss: 1.019641637802124
Epoch 497 / 500 | iteration 10 / 30 | Total Loss: 3.4388134479522705 | KNN Loss: 2.4380390644073486 | BCE Loss: 1.0007743835449219
Epoch 497 / 500 | iteration 15 / 30 | Total Loss: 3.498948335647583 | KNN Loss: 2.4851202964782715 | BCE Loss: 1.0138280391693115
Epoch 497 / 500 | iteration 20 / 30 | Total Loss: 3.4733033180236816 | KNN Loss: 2.4299356937408447 | BCE Loss: 1.0433675050735474
Epoch 497 / 500 | iteration 25 / 30 | Total Loss: 3.4800262451171875 | KNN Loss: 2.467148780822754 | BCE Loss: 1.0128774642944336
Epoch 498 / 500 | iteration 0 / 30 | Total Loss: 3.4869842529296875 | KNN Loss: 2.4547674655914307 | BCE Loss: 1.0322169065475464
Epoch 498 / 500 | iteration 5 / 30 | Total Loss: 3.5106704235076904 | KNN Loss: 2.454357624053955 | BCE Loss: 1.0563127994537354
Epoch 498 / 500 | iteration 10 / 30 | Total Loss: 3.4620797634124756 | KNN Loss: 2.441644191

In [8]:
outputs, iterm = model(dataset[67][0].unsqueeze(0).to(device), return_intermidiate=True)
print(outputs)
print(outputs.min())
print(outputs.max())

tensor([[ 1.8027e+00,  2.0555e+00,  3.2644e+00,  4.4957e+00,  2.6340e+00,
          4.5206e-02,  1.7304e+00,  2.7014e+00,  2.9560e+00,  1.8826e+00,
          2.2371e+00,  2.4518e+00,  1.5381e+00,  1.9312e+00,  1.9301e+00,
          1.6787e+00,  3.6269e+00,  3.7735e+00,  1.8001e+00,  1.6535e+00,
          1.3731e+00,  2.3451e+00,  2.7495e+00,  2.8467e+00,  3.1070e+00,
          1.1494e+00,  2.9233e+00,  1.3491e+00,  1.7394e+00,  5.9677e-01,
         -2.3201e-01,  8.0719e-01,  5.5910e-01,  3.2041e-01,  1.9358e+00,
          1.9745e+00,  1.1563e+00,  1.8492e+00,  1.3308e+00,  1.3106e+00,
          4.0394e-02, -6.2011e-01, -2.3070e-01,  1.8453e+00,  2.2274e+00,
          1.0378e+00,  1.3771e-01, -2.7222e-01,  1.5964e+00,  2.1125e+00,
          1.7411e+00,  5.7037e-01,  1.7317e+00,  6.2665e-01, -5.4628e-01,
          1.6759e+00,  1.3695e+00,  1.7128e+00,  8.5411e-01,  1.7499e+00,
          7.0341e-01,  5.3553e-01,  4.1325e-01,  8.8071e-01,  1.5722e+00,
          1.2705e+00, -1.7034e+00,  4.

In [9]:
plt.figure()
plt.plot(losses)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
dataset.transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [11]:
dataset_ = [d[0].to('cpu') for d in dataset]

In [12]:
model = model.eval().to('cpu')
projections = model.calculate_intermidiate(dataset_)

100%|██████████| 15/15 [00:00<00:00, 17.40it/s]


In [13]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit DBSCAN and calculate indices


In [14]:
clusters = DBSCAN(eps=0.01, min_samples=80).fit_predict(projections)
# scores = []
# best_score = float('inf')
# clusters = None
# range_ = list(range(5, 20))
# for k in tqdm(range_):
#     y = GaussianMixture(n_components=k).fit_predict(projections)
#     cur_score = davies_bouldin_score(projections, y)
#     scores.append(cur_score)
    
#     if cur_score < best_score:
#         best_score = cur_score
#         clusters = y

In [ ]:
perplexity = 100

p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

In [ ]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree
# from sklearn.tree import _tree

In [ ]:
tensor_dataset = torch.stack(dataset_)

In [ ]:
# clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=5)
# clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
# print(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
# print(clf.get_depth())

In [ ]:
# scores = []
# for min_samples in range(1,50, 1):
#     clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=min_samples)
#     clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
#     scores.append(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
    
# plt.figure()
# plt.plot(list(range(1,50, 1)), scores)
# plt.show()

In [ ]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
#             p1 += [f"({name} <= {np.round(threshold, 3)})"]
            p1 += [(name, '<=', np.round(threshold, 3))]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [(name, '>', np.round(threshold, 3))]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = []
        
        for p in path[:-1]:
            rule += [p]
        target = " then "
        if class_names is None:
            target += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            target += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
           
        proba = np.round(100.0*classes[l]/np.sum(classes),2)
        target += f" | based on {path[-1][1]:,} samples"
        rule_wrapper = {'target': target, 'rule': rule, 'proba': proba}
        rules += [rule_wrapper]
        
    return rules

In [ ]:
# rules = get_rules(clf, dataset.items, clusters[clusters != -1])

# for rule in rules:
#     n_pos = 0
#     for c,p,v in rule['rule']:
#         if p == '>':
#             n_pos += 1
#     rule['pos'] = n_pos

In [ ]:
# plt.figure()
# probs = [r['proba'] for r in rules]
# plt.hist(probs, bins = 100)
# plt.show()

In [ ]:
# rules = sorted(rules, key=lambda x:x['pos'])
# rules = [r for r in rules if r['proba'] > 50]
# print(len(rules))

In [ ]:
# for i in range(17):
#     r_i = rules[i]
#     print(f"------------- rule {i} length {len(r_i)} -------------")
#     print(r_i)

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [ ]:
tree_dataset = list(zip(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [ ]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [ ]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [ ]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 100
output_dim = len(set(clusters))
log_interval = 1
use_cuda = device != 'cpu'

In [ ]:
tree = SDT(input_dim=tensor_dataset.shape[1], output_dim=len(clusters - 1), depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [ ]:
losses = []
accs = []
sparsity = []

In [ ]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

In [ ]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

In [ ]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

# Tree Visualization

In [ ]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

# Extract Rules

# Accumulate samples in the leaves

In [ ]:
print(f"Number of patterns: {len(root.get_leaves())}")

In [ ]:
method = 'greedy'

In [ ]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

# Tighten boundaries

In [ ]:
attr_names = dataset.items

# print(attr_names)
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")